In [1]:
import numpy as np
import pptk
import os

def load_points(bin_path):
    """Load point cloud from .bin file"""
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    return points

def load_labels(label_path):
    """Load semantic labels"""
    labels = np.fromfile(label_path, dtype=np.int32)
    # Semantic labels are in the lower 16 bits
    labels = labels & 0xFFFF
    return labels

def get_color_map():
    """Define color map for visualization"""
    color_map = {
        0: [0, 0, 0],        # unlabeled - black
        1: [245, 150, 100],  # car - orange
        2: [245, 230, 100],  # bicycle - yellow
        3: [150, 60, 30],    # motorcycle - brown
        4: [180, 30, 80],    # truck - dark red
        5: [255, 0, 0],      # other-vehicle - red
        6: [30, 30, 255],    # person - blue
        7: [200, 40, 255],   # bicyclist - purple
        8: [90, 30, 150],    # motorcyclist - dark purple
        9: [255, 0, 255],    # road - magenta
        10: [255, 150, 255], # parking - light magenta
        11: [75, 0, 75],     # sidewalk - dark purple
        12: [75, 0, 175],    # other-ground - purple
        13: [0, 200, 255],   # building - light blue
        14: [50, 120, 255],  # fence - blue
        15: [0, 175, 0],     # vegetation - green
        16: [0, 60, 135],    # trunk - dark blue
        17: [80, 240, 150],  # terrain - light green
        18: [150, 240, 255], # pole - light blue
        19: [0, 0, 255]      # traffic-sign - blue
    }
    return color_map

def visualize_frame(sequence_path, frame_number):
    """Visualize a single frame with labels and intensity"""
    # Construct file paths
    points_file = os.path.join(sequence_path, 'velodyne', f'{frame_number:06d}.bin')
    labels_file = os.path.join(sequence_path, 'PRE', f'{frame_number:06d}.label')
    
    # Load point cloud and labels
    points = load_points(points_file)
    labels = load_labels(labels_file)
    
    # Get color map
    color_map = get_color_map()
    
    # Convert labels to colors
    colors = np.zeros((len(labels), 3))
    for label_id, color in color_map.items():
        mask = labels == label_id
        colors[mask] = np.array(color) / 255.0
    
    # Extract intensity values (fourth column of points array)
    intensities = points[:, 3]
    
    # Normalize intensities to [0, 1] range for better visualization
    intensities_normalized = (intensities - intensities.min()) / (intensities.max() - intensities.min())
    
    # Visualize using pptk
    v = pptk.viewer(points[:, :3])  # Only use x, y, z coordinates
    
    # Set both colors and intensities as attributes
    # You can switch between them using 'a' key in the viewer
    v.attributes(colors, intensities_normalized)
    v.set(point_size=0.02)
    
    return v

In [2]:
# Set paths
base_path = r"D:\484_final_project\dataset"  # Change this to your dataset path
sequence = "00"  # Change this to visualize different sequences
frame = 200       # Change this to visualize different frames

sequence_path = os.path.join(base_path, 'sequences', sequence)

# Visualize
viewer = visualize_frame(sequence_path, frame)


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\484_final_project\\dataset\\sequences\\00\\PRE\\000200.label'

In [3]:
import numpy as np
import pptk
import os

def load_points(bin_path):
    """Load point cloud from .bin file"""
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    return points

def load_labels(label_path):
    """Load semantic labels"""
    labels = np.fromfile(label_path, dtype=np.int32)
    # Semantic labels are in the lower 16 bits
    labels = labels & 0xFFFF
    return labels

def get_color_map():
    """Define color map for visualization"""
    color_map = {
        0: [0, 0, 0],        # unlabeled - black
        1: [245, 150, 100],  # car - orange
        2: [245, 230, 100],  # bicycle - yellow
        3: [150, 60, 30],    # motorcycle - brown
        4: [180, 30, 80],    # truck - dark red
        5: [255, 0, 0],      # other-vehicle - red
        6: [30, 30, 255],    # person - blue
        7: [200, 40, 255],   # bicyclist - purple
        8: [90, 30, 150],    # motorcyclist - dark purple
        9: [255, 0, 255],    # road - magenta
        10: [255, 150, 255], # parking - light magenta
        11: [75, 0, 75],     # sidewalk - dark purple
        12: [75, 0, 175],    # other-ground - purple
        13: [0, 200, 255],   # building - light blue
        14: [50, 120, 255],  # fence - blue
        15: [0, 175, 0],     # vegetation - green
        16: [0, 60, 135],    # trunk - dark blue
        17: [80, 240, 150],  # terrain - light green
        18: [150, 240, 255], # pole - light blue
        19: [0, 0, 255]      # traffic-sign - blue
    }
    return color_map

def visualize_frame_range(sequence_path, start_frame, end_frame, frame_step=1):
    """
    Visualize multiple frames together
    
    Args:
        sequence_path (str): Path to the sequence directory
        start_frame (int): Starting frame number
        end_frame (int): Ending frame number (inclusive)
        frame_step (int): Step size between frames (default=1)
    
    Returns:
        pptk.viewer: Viewer object
    """
    all_points = []
    all_labels = []
    frame_offsets = []  # Store frame-specific offsets for visualization
    
    # Load and combine all frames
    for frame_number in range(start_frame, end_frame + 1, frame_step):
        # Construct file paths
        points_file = os.path.join(sequence_path, 'velodyne', f'{frame_number:06d}.bin')
        labels_file = os.path.join(sequence_path, 'predictions', f'{frame_number:06d}.label')
        
        if not (os.path.exists(points_file) and os.path.exists(labels_file)):
            print(f"Skipping frame {frame_number} - files not found")
            continue
            
        # Load point cloud and labels
        points = load_points(points_file)
        labels = load_labels(labels_file)
        
        # Add frame-specific offset to x coordinates to separate frames
        frame_offset = len(frame_offsets) * 5.0  # 5 meters between frames
        points_offset = points.copy()
        points_offset[:, 0] += frame_offset  # Offset x coordinates
        
        all_points.append(points_offset)
        all_labels.append(labels)
        frame_offsets.append(frame_offset)
    
    if not all_points:
        raise ValueError("No valid frames found in the specified range")
    
    # Combine all points and labels
    combined_points = np.vstack(all_points)
    combined_labels = np.concatenate(all_labels)
    
    # Get color map and convert labels to colors
    color_map = get_color_map()
    colors = np.zeros((len(combined_labels), 3))
    for label_id, color in color_map.items():
        mask = combined_labels == label_id
        colors[mask] = np.array(color) / 255.0
    
    # Extract and normalize intensities
    intensities = combined_points[:, 3]
    intensities_normalized = (intensities - intensities.min()) / (intensities.max() - intensities.min())
    
    # Create viewer
    v = pptk.viewer(combined_points[:, :3])
    v.attributes(colors, intensities_normalized)
    v.set(point_size=0.02)
    
    # Set a better view point to see all frames
    center_point = combined_points[:, :3].mean(axis=0)
    v.set(lookat=center_point)
    
    # Print information about visualization
    print(f"Visualizing {len(frame_offsets)} frames from {start_frame} to {end_frame}")
    print("Use 'a' key to switch between color and intensity visualization")
    print("Use mouse to rotate/pan/zoom the view")
    
    return v

def visualize_frame_list(sequence_path, frame_numbers):
    """
    Visualize specific frames by their numbers
    
    Args:
        sequence_path (str): Path to the sequence directory
        frame_numbers (list): List of frame numbers to visualize
    
    Returns:
        pptk.viewer: Viewer object
    """
    if not frame_numbers:
        raise ValueError("Frame numbers list is empty")
    
    start_frame = min(frame_numbers)
    end_frame = max(frame_numbers)
    
    def custom_frame_filter(frame):
        return frame in frame_numbers
    
    # Use visualize_frame_range with step=1 and skip unwanted frames
    all_points = []
    all_labels = []
    frame_offsets = []
    
    for frame_number in range(start_frame, end_frame + 1):
        if not custom_frame_filter(frame_number):
            continue
            
        points_file = os.path.join(sequence_path, 'velodyne', f'{frame_number:06d}.bin')
        labels_file = os.path.join(sequence_path, 'predictions', f'{frame_number:06d}.label')
        
        if not (os.path.exists(points_file) and os.path.exists(labels_file)):
            print(f"Skipping frame {frame_number} - files not found")
            continue
        
        points = load_points(points_file)
        labels = load_labels(labels_file)
        
        frame_offset = len(frame_offsets) * 5.0
        points_offset = points.copy()
        points_offset[:, 0] += frame_offset
        
        all_points.append(points_offset)
        all_labels.append(labels)
        frame_offsets.append(frame_offset)
    
    if not all_points:
        raise ValueError("No valid frames found in the specified list")
    
    combined_points = np.vstack(all_points)
    combined_labels = np.concatenate(all_labels)
    
    color_map = get_color_map()
    colors = np.zeros((len(combined_labels), 3))
    for label_id, color in color_map.items():
        mask = combined_labels == label_id
        colors[mask] = np.array(color) / 255.0
    
    intensities = combined_points[:, 3]
    intensities_normalized = (intensities - intensities.min()) / (intensities.max() - intensities.min())
    
    v = pptk.viewer(combined_points[:, :3])
    v.attributes(colors, intensities_normalized)
    v.set(point_size=0.02)
    
    center_point = combined_points[:, :3].mean(axis=0)
    v.set(lookat=center_point)
    
    print(f"Visualizing {len(frame_offsets)} frames: {sorted(frame_numbers)}")
    print("Use 'a' key to switch between color and intensity visualization")
    print("Use mouse to rotate/pan/zoom the view")
    
    return v

In [ ]:
# Set paths
base_path = r"D:\484_final_project\dataset"  # Change this to your dataset path
sequence = "00"  # Change this to visualize different sequences

sequence_path = os.path.join(base_path, 'sequences', sequence)

# Visualize
# v1 = visualize_frame_range(sequence_path, 0, 5)
predictions, frame_nums = concatenate_predictions(
    sequence_path
)

Processed 767 frames
Output shape: (767, 131072)


In [12]:
predictions.shape

(767, 131072)

In [13]:
np.save('slam03_predictions_01.npy',predictions)

In [10]:
import numpy as np
import os
from glob import glob

def concatenate_predictions(sequence_path):
    """
    Concatenate all prediction labels into a single numpy array.
    
    Args:
        sequence_path (str): Path to the sequence directory containing 'predictions' folder
    
    Returns:
        np.ndarray: Concatenated predictions with shape (num_frames)
        list: List of frame numbers that were processed
    """
    # Get all prediction files
    pred_path = os.path.join(sequence_path, 'predictions_grid_01')
    pred_files = glob(os.path.join(pred_path, '*.label'))
    pred_files.sort()  # Ensure files are in order
    
    num_frames = len(pred_files)
    if num_frames == 0:
        raise ValueError(f"No prediction files found in {pred_path}")
    
    # List to store all predictions
    all_predictions = []
    frame_numbers = []
    
    for pred_file in pred_files:
        # Extract frame number from filename
        frame_num = int(os.path.basename(pred_file).split('.')[0])
        frame_numbers.append(frame_num)
        
        try:
            # Load labels
            labels = np.fromfile(pred_file, dtype=np.int32)
            labels = labels & 0xFFFF  # Get semantic labels from lower 16 bits
            all_predictions.append(labels)
            
        except Exception as e:
            print(f"Error processing frame {frame_num}: {str(e)}")
            continue
    
    # Stack all predictions
    predictions = np.stack(all_predictions)
    
    print(f"Processed {len(frame_numbers)} frames")
    print(f"Output shape: {predictions.shape}")
    
    return predictions, frame_numbers

def save_concatenated_predictions(sequence_path, output_path):
    """
    Concatenate predictions and save to a numpy file.
    
    Args:
        sequence_path (str): Path to the sequence directory
        output_path (str): Path to save the concatenated predictions
    """
    predictions, frame_numbers = concatenate_predictions(sequence_path)
    
    # Save both predictions and frame numbers
    np.savez_compressed(
        output_path,
        predictions=predictions,
        frame_numbers=frame_numbers
    )
    print(f"Saved concatenated predictions to {output_path}")
    
def load_concatenated_predictions(file_path):
    """
    Load previously saved concatenated predictions.
    
    Args:
        file_path (str): Path to the saved .npz file
    
    Returns:
        tuple: (predictions array, frame numbers list)
    """
    data = np.load(file_path)
    return data['predictions'], data['frame_numbers'].tolist()